In [33]:
import gensim.summarization.bm25 
import gensim
import os
import numpy as np
import pickle
import spacy
import torch
import transformers
import pickle
import numpy as np

In [24]:
import bs4
import urllib.request
import re
import timeit
import os
import spacy

class WebsiteData():
    
    def __init__(self, website,depth, pathName):#,website,parser = 'lxml'):
        '''
        Constructor of the class. We let the user choose the parser type, but keep it default to the fastest parser.
        We are keeping the bsObject private because the end user should NOT get access to any HTML information.
        '''
        self.website = website
        self.pageDict = {}
        self.depth = depth
        self.pathName = pathName
        '''
        try:
            sauce = urllib.request.urlopen(self.website)
            self.__bsObject = bs4.BeautifulSoup(sauce,parser)
        except:
            print("Website not accessible : " ,self.website)
            self.__bsObject = None
            return None
        '''
            
    def parseBaseWebsite(self, parser = 'lxml'):
        try:
            sauce = urllib.request.urlopen(self.website)
            self.__bsObject = bs4.BeautifulSoup(sauce,parser)
            return 1
        except Exception as e:
            print("Website not accessible. Exception :  " ,e)
            self.__bsObject = None
            return None
    
    def __getBaseURL(self):
        '''
        Return the base url of the website being parsed
        '''
        count = 0
        base1 = []
        base = ''
        if re.findall('http[s]?', self.website):
            for i in self.website:
                if i == '/':
                    count= count+1
                if count == 3:
                    break
                base1.append(i)
        
            base = base.join(base1)
        
        return base
    
    def displayHtml(self):
        '''
        Display website in HTML Structure 
        '''
        print(self.__bsObject.prettify())
    
    def retrieveHrefs(self):
        '''
        Retrieve links from the HREF Tags in the webpage 
        '''
        self.__linksList = []
        baseUrl = self.__getBaseURL()
        print("base url : ", baseUrl)
        
       
        bodyContent = self.__bsObject.find("div", attrs = {'id': 'mntl-sc-page_1-0'})
        
        
        for link in bodyContent.find_all('a'):
            #print(link)
            suffixURL = link.get('href')
            #print(type(suffixURL))
            print(suffixURL)
            
            if (suffixURL is not None) and ('www.investopedia.com' in suffixURL) and (suffixURL.endswith('.asp'))  :
                self.__linksList.append(str(suffixURL))
                
         
        return self.__linksList
    
    def saveWebsiteData(self, levels):
        '''
        Save the data from the website in a txt file. The method creates a file with the topic of the link as the file name.
        The code will be extended to store the data from all the links in the linksList in the future.
        '''
        pathName = self.pathName
        if levels == 0 :
            pathName = pathName
        else:
            for counter in range(levels):
                pathName = pathName +  "/level" + str(counter+1 )
                
        if not os.path.exists(os.path.join(os.getcwd(),pathName)):
            os.mkdir(os.path.join(os.getcwd(),pathName ))
                

        #headerContent = self.__bsObject.find_all("span", attrs = {'class': 'mntl-sc-block-heading__text'})
        bodyContent = self.__bsObject.find("div", attrs = {'id': 'mntl-sc-page_1-0'})
        paragraphs = bodyContent.find_all('p')
        #headers = headerContent.find_all('')

        print("writing data")        
        file_name = os.path.join(os.path.join(os.getcwd(), pathName), (self.website).split("/")[-1] + ".txt")
        print(file_name)
        with open(file_name ,"w+", encoding='utf-8') as f:
            maxParaLen = 0
            sumParaLen = 0
            nlp = spacy.load("en_core_web_sm",disable=['ner', 'parser', 'textcat'])
            totalParaLen = len(paragraphs)
            for paragraph in paragraphs:
                
                if paragraph.find_parent('div').find_previous('h3') is not None:
                    heading = paragraph.find_parent('div').find_previous('h3')
                    if heading.find('span') is not None:
                        
                        heading_text = heading.find('span').text
                    else:
                        heading_text = heading.text
                else:
                    heading = paragraph.find_parent('div').find_previous('h1')
                    heading_text = heading.text
                    
                 
                paraText = heading_text+ ". " + paragraph.getText()
                paraTextDoc = nlp(paraText)
                paraTextDocList = [tokens.text for tokens in paraTextDoc if tokens.text.strip() != '']
                paraLen = len(paraTextDocList)
                paraText = " ".join(paraTextDocList)
                sumParaLen = sumParaLen + paraLen
                if paraLen > maxParaLen:
                    maxParaLen = paraLen

                f.write(paraText + '\n--------------------------\n')
                

            avgParaLen = (sumParaLen/totalParaLen)
            self.pageDict[(self.website).split("/")[-1]] = (maxParaLen , avgParaLen)

            f.close()
            return self.pageDict
        


class WebsiteParser:
    
    def __init__(self,baseWebsite,level, pathName):
        
        self.baseWebsite = baseWebsite
        self.level = level
        self.pathName = pathName
    
    def parseWebsite(self):
        
        pageDict ={}
        linkDict = {}
        
        for level in range(self.level):
            
            if level == 0:
                obj = WebsiteData(self.baseWebsite,level, self.pathName)
                objret = obj.parseBaseWebsite()
                linkDict[level +1] = obj.retrieveHrefs()
                if objret is not None: 
                    data = obj.saveWebsiteData(level) 
                    pageDict.update(data)
            else:
                linkDict[level +1] = []
        
                if level < (self.level - 1):
                
                    for link in set(linkDict[level]):
                        obj1 = WebsiteData(link, level, self.pathName)
                        obj1ret = obj1.parseBaseWebsite()
                       
                        linkDict[level +1].extend(obj1.retrieveHrefs())
                        
                        if obj1ret is not None: 
                            data = obj1.saveWebsiteData(level) 
                            pageDict.update(data)
                
                else:
                    for link in set(linkDict[level]):
                        obj1 = WebsiteData(link, level, self.pathName)
                        obj1ret = obj1.parseBaseWebsite()
                        
                        if obj1ret is not None: 
                            data = obj1.saveWebsiteData(level) 
                            pageDict.update(data)

In [25]:
numLevels = 3
initLink = 'https://www.investopedia.com/terms/r/riskmanagement.asp'

In [26]:
parser = WebsiteParser(initLink,numLevels,r"C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data")
parser.parseWebsite()

base url :  https://www.investopedia.com
https://www.investopedia.com/terms/f/financialplanner.asp
https://www.investopedia.com/terms/r/risk-analysis.asp
https://www.investopedia.com/ask/answers/042415/what-difference-between-moral-hazard-and-adverse-selection.asp
https://www.investopedia.com/articles/pf/07/risk_tolerance.asp
None
https://www.investopedia.com/terms/u/ustreasury.asp
https://www.investopedia.com/terms/d/derivative.asp
https://www.investopedia.com/terms/o/option.asp
https://www.investopedia.com/terms/f/futures.asp
https://www.investopedia.com/terms/s/subprime-meltdown.asp
https://www.investopedia.com/terms/m/mbs.asp
None
https://www.investopedia.com/terms/b/benchmark.asp
https://www.investopedia.com/terms/v/volatility.asp
None
https://www.investopedia.com/terms/s/standarddeviation.asp
https://www.investopedia.com/terms/d/dispersion.asp
https://www.investopedia.com/terms/a/annualized-total-return.asp
https://www.investopedia.com/terms/s/sp500.asp
None
https://www.investope

base url :  https://www.investopedia.com
None
https://atlas.dotdash.com/terms/l/lender.asp
https://www.investopedia.com/terms/d/default2.asp
https://www.investopedia.com/terms/l/libor.asp
https://atlas.dotdash.com/terms/n/netinterestmargin.asp
https://www.investopedia.com/terms/m/moodys.asp
https://www.investopedia.com/terms/s/sp.asp
None
https://www.investopedia.com/terms/s/syndicate.asp
None
https://www.investopedia.com/terms/m/mergersandacquisitions.asp
https://www.investopedia.com/terms/l/leveragedbuyout.asp
https://www.investopedia.com/terms/d/dividend.asp
None
writing data
C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data/level1\leveragedloan.asp.txt
base url :  https://www.investopedia.com
None
https://www.investopedia.com/terms/m/marketindex.asp
https://www.investopedia.com/markets/quote?tvwidgetsymbol=spy
https://www.investopedia.com/terms/s/sp500.asp
https://www.investopedia.com/terms/m/marketablesecurities.asp
https://www.investopedia.com/ask/answers/09/mutu

base url :  https://www.investopedia.com
None
https://www.investopedia.com/terms/n/null_hypothesis.asp
https://www.investopedia.com/terms/t/type-ii-error.asp
None
https://www.investopedia.com/terms/a/alpha-risk.asp
https://www.investopedia.com/terms/t/type_1_error.asp
https://www.investopedia.com/terms/s/sample.asp
https://www.investopedia.com/terms/v/variance.asp
None
https://www.investopedia.com/terms/h/hypothesistesting.asp
https://www.investopedia.com/terms/z/zscore.asp
None
https://www.investopedia.com/investing/beta-know-risk/
https://www.investopedia.com/terms/c/capm.asp
writing data
C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data/level1\beta-risk.asp.txt
base url :  https://www.investopedia.com
None
https://www.investopedia.com/terms/f/financialinstrument.asp
https://www.investopedia.com/terms/d/derivative.asp
https://www.investopedia.com/ask/answers/difference-between-options-and-futures/
https://www.investopedia.com/terms/c/call.asp
https://www.investopedia

base url :  https://www.investopedia.com
None
https://www.investopedia.com/terms/s/security.asp
https://www.investopedia.com/best-online-brokers-4587872
https://www.investopedia.com/terms/c/counterpartyrisk.asp
https://www.investopedia.com/terms/e/exchange-traded-derivative.asp
None
https://www.investopedia.com/terms/f/foreignexchangerisk.asp
https://www.investopedia.com/terms/c/currencyfuture.asp
https://www.investopedia.com/terms/c/currencyswap.asp
https://www.investopedia.com/ask/answers/070615/what-difference-between-derivatives-and-options.asp
None
https://www.investopedia.com/terms/f/futurescontract.asp
https://www.investopedia.com/terms/s/shortselling.asp
https://www.investopedia.com/terms/i/indexfutures.asp
https://www.investopedia.com/terms/f/forwardcontract.asp
https://www.investopedia.com/terms/i/interestrateswap.asp
https://www.investopedia.com/terms/s/swaprate.asp
https://www.investopedia.com/terms/o/optionscontract.asp
https://www.investopedia.com/terms/s/strikeprice.asp


base url :  https://www.investopedia.com
https://www.investopedia.com/terms/r/risktolerance.asp
None
https://www.investopedia.com/terms/t/timehorizon.asp
https://www.investopedia.com/terms/a/assetclasses.asp
https://www.investopedia.com/terms/c/certificateofdeposit.asp
https://www.investopedia.com/terms/t/treasurybill.asp
None
https://www.investopedia.com/terms/r/riskcapital.asp
https://www.investopedia.com/terms/l/limitedrisk.asp
None
https://www.investopedia.com/terms/d/disposableincome.asp
https://www.investopedia.com/terms/t/taxshelter.asp
https://www.investopedia.com/terms/f/futuresmarket.asp
None
https://www.investopedia.com/terms/p/preservationofcapital.asp
None
https://www.investopedia.com/terms/d/diversification.asp
writing data
C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data/level1\risk_tolerance.asp.txt
writing data
C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data/level1/level2\underlying-asset.asp.txt
writing data
C:\Users\piyush.kathuri

writing data
C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data/level1/level2\full-faith-credit.asp.txt
writing data
C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data/level1/level2\bearmarket.asp.txt
writing data
C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data/level1/level2\zomma.asp.txt
writing data
C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data/level1/level2\irs.asp.txt
writing data
C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data/level1/level2\what-difference-between-paidup-capital-and-share-capital.asp.txt
writing data
C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data/level1/level2\sampling-distribution.asp.txt
writing data
C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data/level1/level2\financial-crisis-review.asp.txt
writing data
C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data/level1/level2\profits-interest.asp.txt
writing data
C:\Users\piy

writing data
C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data/level1/level2\assetallocation.asp.txt
writing data
C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data/level1/level2\russell2000.asp.txt
writing data
C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data/level1/level2\vanguard-total-stock-index-vs-vanguard-500-index-fund.asp.txt
writing data
C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data/level1/level2\sp500.asp.txt
writing data
C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data/level1/level2\benchmark.asp.txt
writing data
C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data/level1/level2\tech_bubble.asp.txt
writing data
C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data/level1/level2\montecarlosimulation.asp.txt
writing data
C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data/level1/level2\purerisk.asp.txt
writing data
C:\Users\piyush.kathuria\Docume

writing data
C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data/level1/level2\oracleofomaha.asp.txt
writing data
C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data/level1/level2\what-difference-between-derivatives-and-options.asp.txt
writing data
C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data/level1/level2\gips.asp.txt
writing data
C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data/level1/level2\gain.asp.txt
writing data
C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data/level1/level2\what-are-some-common-measures-risk-used-risk-management.asp.txt
writing data
C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data/level1/level2\confirmation-bias.asp.txt
writing data
C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data/level1/level2\rho.asp.txt
writing data
C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data/level1/level2\stockmarket.asp.txt
writing data
C:\Users\

writing data
C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data/level1/level2\subprime_mortgage.asp.txt
writing data
C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data/level1/level2\market-value-of-equity.asp.txt
writing data
C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data/level1/level2\what-are-some-examples-moral-hazard-business-world.asp.txt
writing data
C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data/level1/level2\variance.asp.txt
writing data
C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data/level1/level2\currency-etf.asp.txt
writing data
C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data/level1/level2\fiduciary.asp.txt
writing data
C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data/level1/level2\returnonequity.asp.txt
writing data
C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data/level1/level2\delta.asp.txt
writing data
C:\Users\piyush.kathuria\D

In [34]:
class DocumentRanker():
    
    def __init__(self, query, folderPath):
        
        self.documentFolderPath = folderPath
        print("folderPath  : ", folderPath)
        self.query = self.convertQuery(query)
        self.folderDocumentContent = self.loadDocuments()  # Store the content of all the files in dictionary
        self.bm25Model = self.createBM25Model()
    
    
    def convertQuery(self,query):
        
        nlp = spacy.load("en_core_web_sm",disable = ['ner', 'parser', 'textcat'])
        doc = nlp(query.lower())
        return doc
        
    def loadDocuments(self):
        '''
        Load documents from the invest_data folder into a dictionary
        '''
        documentTuple = []
        nounList = self.getNouns()
        nlp = spacy.load("en_core_web_sm",disable = ['ner', 'parser', 'textcat'])
        
        for root, dirs, files in os.walk(self.documentFolderPath):
    
            for file in files:
                if file.endswith('txt'):
                    with open(os.path.abspath(os.path.join(root, file)), 'r' , encoding="utf8" ) as f:
                        content = f.read()
                        content = content.lower()
                        doc = nlp(content)
                        tokenizedContent = [tokens.text for tokens in doc]  
                        documentTuple.append((file,tokenizedContent))
                            
                        #print("selected files : ", file)
                        
        print("THIS IS SPACY VERSION !!!!!")
        return documentTuple
    
    def getNouns(self):
        
        #nlp = spacy.load("en_core_web_sm")
        
        '''
        spacy_stopwords = list(spacy.lang.en.stop_words.STOP_WORDS)
        
        querySplit = self.query.lower().split()
        
        querySplit = [x for x in querySplit if x not in spacy_stopwords]
        
        print(querySplit)
        
        return querySplit
        
        '''
        nounList = []
        for token in self.query:
            #print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop)
            if token.pos_ == "NOUN" or token.pos_ == "PROPN" or token.pos_ == "ADJ":
                nounList.append(token.text)
            
        print("noun list : ", nounList)        
        return nounList
    
        
    def createBM25Model(self):
        '''
        Return a created BM25 model.
        '''
        return gensim.summarization.bm25.BM25([x[1] for x in self.folderDocumentContent])
    
    def rankDocuments(self):
        '''
        Rank the documents in the corpus wrt the query. 
        '''
        query = [tokens.text for tokens in self.query]
        scores = self.bm25Model.get_scores(query)
        documentScores =  [(self.folderDocumentContent[i][0],scores[i]) for i in range(len(scores))]
        return documentScores
    
    def returnTopK(self,k=5):
        '''
        Return top k ranked documents wrt given user query
        '''
        
        scoreDict = self.rankDocuments()
        return sorted(scoreDict , key = lambda x: x[1], reverse = True)[:k]
    
    def returnTopDocuments(self):
        '''
        Return top documents wrt given user query. This uses a different approach from our 
        previous function. We return all documents with score a standard deviation above the
        mean score of our corpus
        '''
        
        scoreTup = self.rankDocuments()
        #scoreDict = dict(scoreTup)
        scores = np.array([x[1] for x in scoreTup])
        #scores = np.array(list(scoreDict.values()))
        meanScore = np.mean(scores)
        sdScore = np.std(scores)
        maxThreshold = meanScore + sdScore
        topScoresDict = [x for x in scoreTup if x[1] > maxThreshold]
        topDocumentScores = sorted(topScoresDict, key = lambda x: x[1], reverse = True)
        
        return topDocumentScores
    
    def returnTopDocumentsData(self):
        '''
        Return data from the top documents retrieved by returnTopDocuments
        '''
        #query = self.query
        topDocumentScores = self.returnTopDocuments()
        print("topDocumentScores  : ", topDocumentScores)
        topDocumentNames = [x[0] for x in topDocumentScores]
        #print(topDocumentNames)
        
        topDocumentText = [x for x in self.folderDocumentContent if x[0] in topDocumentNames]
        return topDocumentText

In [35]:
class PassageRanker():
    
    def __init__(self, query, topDocumentContent):
        
        self.query = self.convertQuery(query)
        self.topDocumentContent = topDocumentContent
        #self.BM25Model = self.createBM25Model()
    
    def convertQuery(self,query):
        
        nlp = spacy.load("en_core_web_sm",disable = ['ner', 'parser', 'textcat'])
        doc = nlp(query.lower())
        return doc
    
    
    def createBM25Model(self,documentContent):
        '''
        Return a created BM25 model.
        '''
        #return gensim.summarization.bm25.BM25([x[1] for x in self.topDocumentContent])
        return gensim.summarization.bm25.BM25(documentContent)
    
    
    def checkNounMatch(self,paragraph,nounList):
        '''
        A function to see if each noun is matching for a paragraph
        '''
        nounBool = True
        
        for nouns in nounList:
            
            paragraphContent = " ".join(paragraph)
            if nouns in paragraphContent:
                nounBool = True
            else:
                nounBool = False
                break
        
        return nounBool
    
    
    def getNouns(self):
        
        #nlp = spacy.load("en_core_web_sm")
        
        '''
        spacy_stopwords = list(spacy.lang.en.stop_words.STOP_WORDS)
        
        querySplit = self.query.lower().split()
        
        querySplit = [x for x in querySplit if x not in spacy_stopwords]
        
        print(querySplit)
        
        return querySplit
        
        '''
        nounList = []
        for token in self.query:
            #print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop)
            if token.pos_ == "NOUN" or token.pos_ == "PROPN" or token.pos_ == "ADJ":
                nounList.append(token.text)
            
        #print("noun list : ", nounList)        
        return nounList
    
    def returnParagraphList(self,content):
        '''
        Return pargaraph in the form of list of lists to be fed to the BM25 Model
        '''
        paragraphList = []
        sepCounter = 0
        nounList = self.getNouns()
        for i,term in enumerate(content):
            
            if term == "--------------------------":
                paragraph = content[sepCounter:i] # " ".join() <-
                if self.checkNounMatch(self,paragraph,nounList):
                    paragraphList.append(paragraph)
                
                sepCounter = i+1
        
        return paragraphList
    
    
    def returnParagraphScores(self,bm25Model,query):
        '''
        Return the score of a paragraph given a model.
        '''
        return bm25Model.get_scores(query)
        
    
    def returnTopPassages(self, k=10):
        '''
        Rank each paragraph from the document and return the top 10 passages from the collection of documents
        '''
        query = [tokens.text for tokens in self.query]
        
        #documentParagraphScores = [(document[0],self.rankPassages(document[1])) for document in topDocumentContent]
        paragraphScoreTup = []
        paragraphLoL = []
        for documents in self.topDocumentContent:
            #print(documents)
            paragraphLoL.extend(self.returnParagraphList(documents[1]))
        
        
        print(len(paragraphLoL))
        passageBM25Model = self.createBM25Model(paragraphLoL)
        
        #for documents in self.topDocumentContent:
        paragraphScores = self.returnParagraphScores(passageBM25Model, query)
        for i,paragraph in enumerate(paragraphLoL):
                
            paragraphScoreTup.append((paragraph,paragraphScores[i]))
        
        print(" length of paragraphScoreTup : ", len(paragraphScoreTup))
        
        self.topParagraphScores = sorted(paragraphScoreTup, key = lambda x: x[1], reverse = True)
        
        return self.topParagraphScores#[:k]

In [36]:
class bertQAModel():
    
    def __init__(self):
        
        self.tokenizerModel,self.bertQAModel = self.__initializeModel()
    
    def __initializeModel(self):
        '''
        Initialize the Beret Tokenizer and the QA model. Note that this is currently compatible with
        transformers module, NOT pytorch/tensorflow
        '''
        with open(r"C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\models\bertTokenizer.pkl","rb") as f:
            tokenizerModel = pickle.load(f)
        
        with open(r"C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\models\bertQAModel.pkl","rb") as f:
            bertQAModel = pickle.load(f)
        
        return tokenizerModel, bertQAModel
    
    def stringProcess(self,answer):
        answerSplit = answer.split(" ##")
        return "".join(answerSplit)
    
    def predict(self,text,question):
        '''
        Predict the answer given a passage and a question.
        '''
        input_text = "[CLS] " + question + " [SEP] " + text + " [SEP]"
        #print("INPUT_TEXT : ")
        #print(input_text)
        input_ids = self.tokenizerModel.encode(input_text)
        #print("TOKENIZED TEXT : ")
        #print(input_ids)
        token_type_ids = [0 if i <= input_ids.index(102) else 1 for i in range(len(input_ids))]
        #print("TOKEN TYPE IDS : ")
        #print(token_type_ids)
        start_scores, end_scores = self.bertQAModel(torch.tensor([input_ids]), token_type_ids=torch.tensor([token_type_ids]))
        all_tokens = self.tokenizerModel.convert_ids_to_tokens(input_ids)
        answer = ' '.join(all_tokens[torch.argmax(start_scores) : torch.argmax(end_scores)+1])
        startScoreInd,endScoreInd = torch.argmax(start_scores),torch.argmax(end_scores)
        startScoreMax, endScoreMax = start_scores.detach().numpy().flatten()[startScoreInd] , end_scores.detach().numpy().flatten()[endScoreInd]
        avgScore = float(np.absolute(startScoreMax) + np.absolute(endScoreMax))
        return (self.stringProcess(answer),avgScore)

if __name__ == "__main__" :
    '''
    obj = bertQAModel()
    obj.predict("Narendar Modi said that there is a lack of good employment opportunities in India","What did Narendar Modi say?")
    '''
    pass

In [37]:
path = r"C:\Users\piyush.kathuria\Desktop\invest-data"

In [38]:
query = "What is a mortgage backed security?"

In [39]:
docRanker = DocumentRanker(query,path)
topDocuments = docRanker.returnTopDocumentsData()

folderPath  :  C:\Users\piyush.kathuria\Desktop\invest-data
noun list :  ['mortgage', 'security']
THIS IS SPACY VERSION !!!!!
topDocumentScores  :  [('mbs.asp.txt', 16.884939303450555), ('fannie-mae-freddie-mac-credit-crisis.asp.txt', 15.487748268427179), ('asset-backedsecurity.asp.txt', 14.614215767426849), ('tranches.asp.txt', 14.350163947990598), ('full-faith-credit.asp.txt', 13.37974048237066), ('how-standard-deviation-used-determine-risk.asp.txt', 11.566890042533027), ('commercialbank.asp.txt', 11.289750011700193), ('great-recession.asp.txt', 11.093410418633106), ('moodys.asp.txt', 10.950489336670394), ('risk_tolerance.asp.txt', 10.67334225760555), ('federal-reserve-note.asp.txt', 10.62737647101863), ('target-rate.asp.txt', 10.561737816247849), ('subprime_mortgage.asp.txt', 10.233257057153882), ('basics-options-profitability.asp.txt', 10.231537769621152), ('subprime-meltdown.asp.txt', 10.135965140846606), ('subprime-meltdown.asp.txt', 10.135965140846606), ('secondmortgage.asp.txt'

In [40]:
passageRanker = PassageRanker(query , topDocuments)
passageRanking = passageRanker.returnTopPassages()

TypeError: checkNounMatch() takes 3 positional arguments but 4 were given

In [ ]:
passageRanking

In [ ]:
bertQA = bertQAModel()
answerTuple = []
for passage,_ in passageRanking : 
            
    paraText = " ".join(passage)
    answerTuple.append(bertQA.predict(paraText, query))

In [ ]:
print("ANSWERS : ")
sortedAnswers = sorted(answerTuple, key = lambda x: x[1], reverse = True)

for item in sortedAnswers:
    print(item)

ANSWERS : 
('the homes themselves', 11.520589828491211)
('an investment similar to a bond', 11.202780723571777)
('a type of asset - backed security', 9.790294647216797)
('mbs ) . an mbs is made of multiple mortgage pools that have a wide variety of loans', 9.093761444091797)
('turns the bank into a middleman between the homebuyer and the investment industry', 8.733844757080078)
('assets that lead to bubbles', 6.128958702087402)
('mortgage - related security or a mortgage pass - through', 5.813867568969727)
('a financial security such as a bond or note which is collateralized by a pool of assets such as loans , leases , credit card debt , royalties , or receivables . for investors , asset - backed securities are an alternative to investing in corporate debt . an abs is similar to a mortgage - backed security , except that the underlying securities are not mortgage - based', 5.230574607849121)
('mortgage security meltdown', 3.564000129699707)
('lending money to home buyers', 3.3792233467